# 🚂 Railway Sleeper Detection & Classification
**Train YOLOv8 Detection + ResNet Classification Models**

This notebook trains:
1. **YOLO Detector** - Finds sleepers in images (bounding boxes)
2. **ResNet Classifier** - Classifies condition (Good/Fair/Poor)

---

## 📦 Install Dependencies

In [ ]:
!pip install ultralytics -q
!pip install albumentations -q

In [ ]:
import os
import shutil
from pathlib import Path
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 📂 Dataset Setup
The Kaggle dataset `wachyuutami/concrete-sleepers` has:
- `data train/`: Good class (200), Fair Class (250), Poor class (250)
- `data test/`: Good class (100), Fair Class (100), Poor class (100)

In [ ]:
# Dataset path on Kaggle
DATASET_PATH = "/kaggle/input/concrete-sleepers/concrete railway sleepers"

# Verify dataset exists
if os.path.exists(DATASET_PATH):
    print("✅ Dataset found!")
    for item in os.listdir(DATASET_PATH):
        print(f"  - {item}")
else:
    print("❌ Dataset not found. Make sure to add the dataset to your notebook.")

## 🔧 Part 1: Prepare YOLO Detection Dataset
Convert classification images to detection format with full-image bounding boxes

In [ ]:
# Create YOLO dataset structure
YOLO_DATASET = Path("/kaggle/working/sleeper_yolo_dataset")

for split in ['train', 'valid']:
    (YOLO_DATASET / split / 'images').mkdir(parents=True, exist_ok=True)
    (YOLO_DATASET / split / 'labels').mkdir(parents=True, exist_ok=True)

print("✅ YOLO dataset directories created")

In [ ]:
import random

def convert_to_yolo_format(input_dir, output_dir, split='train', val_ratio=0.2):
    """
    Convert classification dataset to YOLO detection format.
    Since sleeper fills most of the frame, use full-image bounding box.
    """
    all_images = []
    
    # Collect all images from all class folders
    for class_folder in Path(input_dir).iterdir():
        if not class_folder.is_dir():
            continue
        for img_file in class_folder.glob('*.jpg'):
            all_images.append((img_file, class_folder.name))
    
    # Shuffle and split
    random.seed(42)
    random.shuffle(all_images)
    
    split_idx = int(len(all_images) * (1 - val_ratio))
    train_images = all_images[:split_idx]
    val_images = all_images[split_idx:]
    
    def process_split(images, split_name):
        for img_path, class_name in images:
            # Copy image
            new_name = f"{class_name.replace(' ', '_')}_{img_path.name}"
            dst_img = output_dir / split_name / 'images' / new_name
            shutil.copy(img_path, dst_img)
            
            # Create YOLO label (class 0, full image bbox)
            lbl_name = Path(new_name).stem + '.txt'
            dst_lbl = output_dir / split_name / 'labels' / lbl_name
            with open(dst_lbl, 'w') as f:
                f.write('0 0.5 0.5 0.95 0.95\n')
        
        return len(images)
    
    train_count = process_split(train_images, 'train')
    val_count = process_split(val_images, 'valid')
    
    return train_count, val_count

# Convert train data
train_path = f"{DATASET_PATH}/data train"
train_count, val_count = convert_to_yolo_format(train_path, YOLO_DATASET, val_ratio=0.15)

print(f"✅ YOLO dataset created:")
print(f"   Train: {train_count} images")
print(f"   Valid: {val_count} images")

In [ ]:
# Create data.yaml for YOLO training
yaml_content = f"""# Sleeper Detection Dataset
path: {YOLO_DATASET}
train: train/images
val: valid/images

nc: 1
names: ['sleeper']
"""

with open(YOLO_DATASET / 'data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created")
print(yaml_content)

## 🎯 Part 2: Train YOLO Sleeper Detector

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 medium model (better accuracy)
model = YOLO('yolov8m.pt')  # yolov8m for better accuracy on Kaggle GPU

print("✅ YOLOv8m model loaded")

In [ ]:
# Train the model
results = model.train(
    data=str(YOLO_DATASET / 'data.yaml'),
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    device=0,  # Use GPU
    project='/kaggle/working/runs',
    name='sleeper_detector',
    exist_ok=True,
    # Augmentation
    augment=True,
    mosaic=1.0,
    mixup=0.1,
    # Save best model
    save=True,
    plots=True
)

In [ ]:
# Display training results
from IPython.display import Image as IPyImage, display

results_dir = Path('/kaggle/working/runs/sleeper_detector')

# Show confusion matrix
if (results_dir / 'confusion_matrix.png').exists():
    display(IPyImage(filename=str(results_dir / 'confusion_matrix.png')))

# Show results
if (results_dir / 'results.png').exists():
    display(IPyImage(filename=str(results_dir / 'results.png')))

In [ ]:
# Validate and get metrics
best_model = YOLO('/kaggle/working/runs/sleeper_detector/weights/best.pt')
metrics = best_model.val()

print("\n📊 YOLO Sleeper Detection Results:")
print(f"   mAP@50: {metrics.box.map50:.3f}")
print(f"   mAP@50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.p[0]:.3f}")
print(f"   Recall: {metrics.box.r[0]:.3f}")

## 🧠 Part 3: Train ResNet Sleeper Classifier
Classify sleeper conditions: Good / Fair / Poor

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Data transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(f"{DATASET_PATH}/data train", transform=train_transforms)
val_dataset = datasets.ImageFolder(f"{DATASET_PATH}/data test", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Classes: {train_dataset.classes}")
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")

In [ ]:
# Load pretrained ResNet50
model_resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Modify final layer for 3 classes (Good, Fair, Poor)
num_classes = len(train_dataset.classes)
model_resnet.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model_resnet.fc.in_features, num_classes)
)

model_resnet = model_resnet.to(device)
print(f"✅ ResNet50 loaded with {num_classes} output classes")

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_resnet.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

# Training loop
NUM_EPOCHS = 30
best_acc = 0
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in range(NUM_EPOCHS):
    # Training
    model_resnet.train()
    train_loss = 0
    train_correct = 0
    
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS}'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model_resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()
    
    train_loss /= len(train_loader)
    train_acc = train_correct / len(train_dataset)
    
    # Validation
    model_resnet.eval()
    val_loss = 0
    val_correct = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).sum().item()
    
    val_loss /= len(val_loader)
    val_acc = val_correct / len(val_dataset)
    
    scheduler.step(val_loss)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
    
    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model_resnet.state_dict(), '/kaggle/working/sleeper_classifier_best.pt')
        print(f'  ✅ Saved best model (acc={best_acc:.4f})')

print(f"\n🎉 Best Validation Accuracy: {best_acc:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()

ax2.plot(history['train_acc'], label='Train Acc')
ax2.plot(history['val_acc'], label='Val Acc')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy')
ax2.legend()

plt.tight_layout()
plt.savefig('/kaggle/working/training_history.png')
plt.show()

## 💾 Part 4: Save Models for Deployment

In [ ]:
import json

# Create output directory
output_dir = Path('/kaggle/working/models')
output_dir.mkdir(exist_ok=True)

# Copy YOLO model
yolo_src = Path('/kaggle/working/runs/sleeper_detector/weights/best.pt')
if yolo_src.exists():
    shutil.copy(yolo_src, output_dir / 'sleeper_yolo_best.pt')
    print("✅ YOLO model saved")

# Copy ResNet model
resnet_src = Path('/kaggle/working/sleeper_classifier_best.pt')
if resnet_src.exists():
    shutil.copy(resnet_src, output_dir / 'sleeper_resnet_best.pt')
    print("✅ ResNet classifier saved")

# Save model info
model_info = {
    'yolo': {
        'model': 'yolov8m',
        'classes': ['sleeper'],
        'input_size': 640,
        'file': 'sleeper_yolo_best.pt'
    },
    'resnet': {
        'model': 'resnet50',
        'classes': train_dataset.classes,
        'input_size': 224,
        'file': 'sleeper_resnet_best.pt',
        'best_accuracy': best_acc
    }
}

with open(output_dir / 'model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("\n📦 Models saved to /kaggle/working/models/")
for f in output_dir.iterdir():
    print(f"  - {f.name} ({f.stat().st_size / 1024 / 1024:.1f} MB)")

In [ ]:
# Test detection on sample image
from ultralytics import YOLO

# Load trained model
detector = YOLO('/kaggle/working/models/sleeper_yolo_best.pt')

# Get a test image
test_img_path = list(Path(f"{DATASET_PATH}/data test/Good Class").glob('*.jpg'))[0]

# Run detection
results = detector(test_img_path, conf=0.25)

# Show result
results[0].show()

## 📥 Download Models
After training, download the models from `/kaggle/working/models/` and upload to your EC2 server.

In [ ]:
print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)
print("\nModels saved to /kaggle/working/models/")
print("\nTo use on EC2:")
print("1. Download sleeper_yolo_best.pt and sleeper_resnet_best.pt")
print("2. Upload to ~/Code_on_track/railway-yolo-detection/models/")
print("3. Restart the backend server")